In [2]:
import pandas as pd

hate = pd.read_csv("all_examples_hate_raw.csv", sep = "\t")

In [3]:
hate["guideline"]

0                         filth
1                         filth
2                         filth
3                         filth
4                         filth
                 ...           
4715    contempt shouldnt exist
4716    contempt shouldnt exist
4717    contempt shouldnt exist
4718    contempt shouldnt exist
4719    contempt shouldnt exist
Name: guideline, Length: 4720, dtype: object

In [1]:
def process_df(df):
    # Group by 'guideline' and count the number of rows without 'gpt' in 'dataset'
    counts = df[df['dataset'] != 'gpt'].groupby('guideline').size()

    # Filter rows
    for guideline, count in counts.items():
        if count >= 50:
            # Remove rows with 'gpt'
            df = df[~((df['guideline'] == guideline) & (df['dataset'] == 'gpt'))]
        else:
            # Reduce the number of rows including 'gpt' to 50
            total_to_keep = 50
            non_gpt_count = df[(df['guideline'] == guideline) & (df['dataset'] != 'gpt')].shape[0]
            gpt_to_keep = total_to_keep - non_gpt_count
            # If there are already less than 50 rows, keep them as is
            if gpt_to_keep > 0:
                # Randomly select gpt rows to keep
                gpt_rows = df[(df['guideline'] == guideline) & (df['dataset'] == 'gpt')]
                gpt_rows_kept = gpt_rows.sample(min(gpt_to_keep, len(gpt_rows)), random_state=1)
                # Keep only the selected gpt rows and all non-gpt rows
                df = df[(df['guideline'] != guideline) | (df['dataset'] != 'gpt') | (df.index.isin(gpt_rows_kept.index))]
    
    return df

processed_hate = process_df(hate)
processed_hate_file = "all_examples_hate.csv"

processed_hate.to_csv(processed_hate_file, sep="\t", index=False)

NameError: name 'hate' is not defined

In [6]:
processed_hate

,example_id,sentence,dataset,index,guideline,votes,labelers
0,31,wommennnnnn are dirt,dynahate,4838,filth,1;1;1,10;11;2
1,34,ggggggggayyyyyyyyys r dirrrrrrrrrrrtttttt,dynahate,4873,filth,1;1;1,10;11;2
2,56,w o m e n a r e t r a s h,dynahate,14286,filth,1;1;1,10;11;2
3,58,trrrrrrrassssshhh = bbbblllllllllackkkkkkkks,dynahate,11042,filth,1;1;1,10;11;2
4,59,trans people are filth! look in the mirror!,dynahate,4707,filth,1;1;1;1,10;11;2;1
...,...,...,...,...,...,...,...
4645,12154,It is not COVID-19 but #WuhanVirus,asonam,1237945541989826565,spread virus,1;1,2;4
4646,12155,@HuXijin_GT Stay hygienic in your eating habit...,asonam,1246454364706766848,spread virus,1;1,2;4
4647,12157,@Gailyfleur @nytimes Communism created wet mar...,asonam,1246003252480618496,spread virus,1;1,2;4
4648,12158,@CMOMaharashtra Why dont ya say something abou...,asonam,1246375528858128385,spread virus,1;1,2;4
